<a href="https://colab.research.google.com/github/VesalAhsani/Driver-behavior-detection/blob/main/DMS_YOLOv8s_CLS_224_fullAugmented.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Check if the dataset zip file exists in Google Drive
import os
dataset_path = '/content/drive/MyDrive/Split_mixed_newaugmented_224.zip'  # Adjust the path if needed
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"The dataset file at {dataset_path} was not found!")
print("Dataset zip file exists.")

Dataset zip file exists.


In [3]:
# Unzipping the dataset
import zipfile
extract_path = '/content/Split_mixed_newaugmented_224'
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Dataset unzipped successfully!")

Dataset unzipped successfully!


In [4]:
# Check the actual folder structure after unzipping
dataset_actual_path = os.path.join(extract_path, "Split_224")  # Adjust to the nested folder
if not os.path.exists(os.path.join(dataset_actual_path, "train")):
    raise FileNotFoundError("Train folder not found in the dataset path!")

In [5]:
# Install the YOLOv8 library
!pip install ultralytics --upgrade

# Import the YOLO library
from ultralytics import YOLO
print("Ultralytics library imported successfully!")

# Verify the dataset structure
from ultralytics.data.utils import check_cls_dataset
dataset_info = check_cls_dataset(dataset_actual_path)
print(f"Dataset verified: {dataset_info}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.5/974.5 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [6]:
# Count the number of training and validation images
import glob
train_images = glob.glob(os.path.join(dataset_actual_path, "train", "*", "*.jpg"))
val_images = glob.glob(os.path.join(dataset_actual_path, "val", "*", "*.jpg"))

if not train_images or not val_images:
    raise ValueError("Training or validation images were not found. Check dataset paths.")
print(f"Number of training images: {len(train_images)}")
print(f"Number of validation images: {len(val_images)}")

Number of training images: 156504
Number of validation images: 17276


In [7]:
# Define paths
DATASET_PATH = dataset_actual_path  # Path to the unzipped dataset
OUTPUT_PATH = '/content/drive/MyDrive/yolov8s_cls_results'

# Initialize the YOLOv8 model
model = YOLO('yolov8s-cls.pt')  # Using yolov8s-cls as a starting point
print("YOLOv8 model initialized with pretrained weights.")

# Train the model
model.train(
    data=DATASET_PATH,  # Path to dataset
    epochs=50,          # Adjust based on your needs
    batch=32,           # Adjust according to GPU memory
    imgsz=224,          # Image size, as per your dataset
    project=OUTPUT_PATH,  # Save results to your Google Drive
    name='yolov8s_cls_224',  # Run name
    pretrained=True,     # Start from pretrained weights
    optimizer='Adam',    # Optimizer
    lr0=0.001            # Initial learning rate
)
print("Training completed!")

# Evaluate the model
results = model.val()
print("Validation completed!")

# Save the best weights
best_weights_path = f"{OUTPUT_PATH}/yolov8s_cls_224/weights/best.pt"
os.makedirs(f"{OUTPUT_PATH}/yolov8s_cls_224/weights", exist_ok=True)
!cp /content/runs/classify/yolov8s_cls_224/weights/best.pt {best_weights_path}
print(f"Best weights saved to {best_weights_path}")

100%|██████████| 12.3M/12.3M [00:00<00:00, 92.1MB/s]


YOLOv8 model initialized with pretrained weights.
Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=/content/Split_mixed_newaugmented_224/Split_224, epochs=50, time=None, patience=100, batch=32, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8s_cls_results, name=yolov8s_cls_224, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=

100%|██████████| 5.35M/5.35M [00:00<00:00, 64.0MB/s]


AMP: checks passed ✅


train: Scanning /content/Split_mixed_newaugmented_224/Split_224/train... 156504 images, 0 corrupt: 100%|██████████| 156504/156504 [00:48<00:00, 3252.66it/s]


...


50 epochs completed in 2.581 hours.
Optimizer stripped from /content/drive/MyDrive/yolov8s_cls_results/yolov8s_cls_224/weights/last.pt, 10.3MB
Optimizer stripped from /content/drive/MyDrive/yolov8s_cls_results/yolov8s_cls_224/weights/best.pt, 10.3MB

Validating /content/drive/MyDrive/yolov8s_cls_results/yolov8s_cls_224/weights/best.pt...
Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLOv8s-cls summary (fused): 30 layers, 5,093,134 parameters, 0 gradients, 12.5 GFLOPs
train: /content/Split_mixed_newaugmented_224/Split_224/train... found 156504 images in 14 classes ✅ 
val: /content/Split_mixed_newaugmented_224/Split_224/val... found 17276 images in 14 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 270/270 [00:06<00:00, 41.44it/s]


                   all      0.986      0.999
Speed: 0.1ms preprocess, 0.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/drive/MyDrive/yolov8s_cls_results/yolov8s_cls_224
Training completed!
Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLOv8s-cls summary (fused): 30 layers, 5,093,134 parameters, 0 gradients, 12.5 GFLOPs
train: /content/Split_mixed_newaugmented_224/Split_224/train... found 156504 images in 14 classes ✅ 
val: /content/Split_mixed_newaugmented_224/Split_224/val... found 17276 images in 14 classes ✅ 
test: None...


val: Scanning /content/Split_mixed_newaugmented_224/Split_224/val... 17276 images, 0 corrupt: 100%|██████████| 17276/17276 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 540/540 [00:07<00:00, 71.95it/s]


                   all      0.986      0.999
Speed: 0.1ms preprocess, 0.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/drive/MyDrive/yolov8s_cls_results/yolov8s_cls_2242
Validation completed!
cp: cannot stat '/content/runs/classify/yolov8s_cls_224/weights/best.pt': No such file or directory
Best weights saved to /content/drive/MyDrive/yolov8s_cls_results/yolov8s_cls_224/weights/best.pt
